In [10]:
import os
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import decomposition, cluster

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import polyglot
import math
from polyglot.text import Text, Word

In [14]:

__file__ = os.getcwd()
dirname = os.path.dirname(__file__)
input_file = os.path.join(dirname, 'scripts\\data\\dogodki100_strippedOnlySlov.csv')
df = pd.read_csv(input_file, header = 0)
original_headers = list(df.columns.values)
data_opis_normalized = df['opis'].astype('U')



def get_top_keywords(data, clusters, labels, n_terms):
    df = pd.DataFrame(data).groupby(clusters).mean()
    
    for i,r in df.iterrows():
        print('Cluster {}: '.format(i) + ', '.join([labels[t] for t in np.argsort(r)[-n_terms:]]))

In [15]:


no_features = 2000
vectorizer = TfidfVectorizer(use_idf=True, max_df=0.95, min_df=2, max_features=no_features)  #stop_words='english', 
X_idf = vectorizer.fit_transform(data_opis_normalized)


# agglo = cluster.FeatureAgglomeration(n_clusters=100)
# agglo.fit(X_idf.todense())
# X_reduced = agglo.transform(X_idf.todense())
# X_reduced = X_idf.todense()


# We train the PCA on the dense version of the tf-idf. 
pca = PCA(n_components=2)
X = pca.fit_transform(X_idf.todense())

n_clusters = 7

# Initialize the clusterer with n_clusters value
clusterer = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1, random_state=1) # random_state = 1 (seed)
cluster_labels = clusterer.fit_predict(X)
centers = clusterer.cluster_centers_

def distance(p0, p1):
    return math.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2)

centerDistances = []
for i in range(len(cluster_labels)):
    centerDistances.append(distance(centers[cluster_labels[i]], X[i]))  # distanca med centrom gruče in dogodkom

df['cluster'] = cluster_labels
df['center_distance'] = centerDistances

In [16]:
for i in range(n_clusters):
    print('Cluster ' + str(i) + ':')
    loc_clusterDF = df.loc[df['cluster'] == i]
    loc_clusterDF_sorted = loc_clusterDF.sort_values(by=['cluster', 'center_distance'])
    display(loc_clusterDF_sorted[:20])

Cluster 0:


,naziv,opis,datum_od,stevilo,sifra_prizorisca,cluster,center_distance
1800,Oddrsajmo v 2020,lahek reklo drsališče mesten trg slovenski kon...,2019-12-05 18:00:00,107,7.099263e+14,0,0.001788
180,"Kraljestvo Nabatejcev: Petra, Wadi Rum, Madain...",vabiti potopisen degustacija potovanje jordani...,2020-03-03 19:00:00,121,4.247062e+14,0,0.002393
5217,ROGLA Freestyle Week 2020,pripravljen roči freesyti week glede zima stra...,2020-03-19 09:00:00,611,1.628852e+14,0,0.002828
5760,PUSTOLANDIJA ★ Otroški pustni festival ★ Gospo...,pustolandija otroški pusten festival gospodars...,2020-02-22 09:00:00,1309,2.146718e+14,0,0.002973
1282,Mušje dirke v Levpi 2019,šktd levp vaba mušji dirka levpa nedelja avgus...,2019-08-04 16:00:00,210,1.101644e+15,0,0.003169
3506,"Martinovanje v Ormožu 2019 - Sobota, 9.11",martinovanje ormož oktober končevati trgatev d...,2019-11-09 08:00:00,132,2.405647e+14,0,0.003170
1718,Astronomsko opazovanje neba v Mirnu,lani letos organizirati javen opazovanje nebo ...,2019-08-12 21:00:00,151,1.388677e+15,0,0.003205
65,Lajbah praznuje,sobota februar potekel natanko leto odpreti vr...,2020-02-15 16:00:00,210,9.043886e+14,0,0.003778
295,4. RUNDA NLP Lige 2.0 - [Bolder SCENA],preostal sklepen dejanje letošnji nlp liga run...,2020-03-07 10:00:00,118,2.274036e+14,0,0.004016
4864,Preventivni dogodek Reši življenje,ozaveščanje združiti zavod reševalen pas agenc...,2019-08-10 08:00:00,115,2.654092e+14,0,0.004048


Cluster 1:


,naziv,opis,datum_od,stevilo,sifra_prizorisca,cluster,center_distance
3587,Čubed pod borovci - akustični večer,vabiti četrtek avgust borovec vas kubed kjer v...,2019-08-08 21:00:00,143,1.120437e+14,1,0.003692
2318,LUKA BASI,velik koncert dan žena krilo ljubezen možnost ...,2020-03-04 20:00:00,1359,1.123170e+14,1,0.003815
5867,Pearl Jam Tribute – Jeremy's Alive @Bunker,bunker bunker vračati peareti jama tribut jere...,2020-04-04 21:30:00,201,5.266410e+14,1,0.004314
1746,Glasbeni abonma HkC | Leon Firšt: Moj glasbeni...,četrtek naroden dom celj glasben abon hkca leo...,2019-09-19 19:30:00,168,1.936586e+14,1,0.004829
4728,Poletni glasbeni večeri v Hangarju: Jakob Kobal,jakobov glasba lebdeti umirjen sproščen zvok d...,2019-07-23 20:00:00,247,9.594483e+14,1,0.006165
7576,New Swing Quartet & Komorni orkester NOVA / 1....,new swing quarteti komoren orkester nov novole...,2020-01-01 20:00:00,106,5.079160e+14,1,0.007979
6017,"123 Raps VOL 2. (Mrigz'n'Ghet, Fankadelik, Mir...",glasben društvo raps vol ponoven poletje poslo...,2019-09-14 18:00:00,378,3.120721e+14,1,0.008002
3632,KOALA VOICE izštekani,lani kavka maj zažgati kakopaka zato letos zak...,2019-09-05 20:30:00,325,9.286913e+10,1,0.008125
1636,"Rap Domačica: Puna kuča w / Emkej, Nite, Tropski",rapa domačica emkej urban večer hip hop scena ...,2019-10-11 23:00:00,436,6.401134e+14,1,0.008764
5541,Hindi Zahra // 26.9 // Musicology Barcaffè Ses...,slovenski zgodba serija koncert musicology bar...,2019-09-26 21:00:00,2158,4.395632e+10,1,0.009604


Cluster 2:


...

,naziv,opis,datum_od,stevilo,sifra_prizorisca,cluster,center_distance
3387,Novo leto na Trgu francoske revolucije,december trg francoski revolucija večer barvat...,2019-12-30 19:00:00,801,4.184348e+14,4,0.001883
5744,Zimska pravljica v Višnji Gori,zimski pravljica višnji gora dodaten koncert v...,2019-12-22 20:00:00,261,1.150587e+14,4,0.002452
1190,Zeleno sonce 124: Croatian Beat,soulbrothra slipmati hrvatski funka delegacija...,2019-10-11 23:00:00,165,3.560087e+14,4,0.002586
6497,Christmas Salsastica Party,srečen sals party petek pričakovati odprt rok ...,2019-12-20 22:00:00,372,3.524581e+14,4,0.002760
3437,Vinske Brbončice & ROCK'n ROLL,rezervirati datum vinski brbončica tokrat pred...,2020-01-18 18:00:00,227,3.707759e+14,4,0.002964
4664,Aprilski Master Vinyl,aprilski master vinyti pričakovati ambasada ga...,2020-04-04 21:00:00,230,1.078772e+14,4,0.004395
4483,Danse Macabre // MC Pekarna,vesel december vrata mesec brezglav zlagati sr...,2019-12-21 22:00:00,138,6.158331e+14,4,0.004916
3235,Božično novoletni koncert Pihalnega orkestra V...,član pihalen orkestra vrhpolje tradicionalen p...,2019-12-21 19:00:00,103,1.122963e+14,4,0.005113
1254,"Dražen Zečić & Marko Škugor - Novo mesto, 20.1...",letošnji december posebej pravljičen vrač razp...,2019-12-20 19:00:00,2132,4.890986e+14,4,0.005531
6223,10. Salon Traminec,pripravljen vinskokulinaričen spektakel zdravi...,2019-08-26 17:00:00,732,1.935266e+14,4,0.005871


Cluster 5:


,naziv,opis,datum_od,stevilo,sifra_prizorisca,cluster,center_distance
1779,Muzikal Nune v akciji! | Studenec,poleten gledališče studenec predstava najti ko...,2019-08-23 20:30:00,225,2.036513e+14,5,0.002080
7172,Večer črnega humorja: Black magic,črn humor prihajati nov mesto skupaj neprimere...,2019-10-25 21:00:00,620,1.895903e+14,5,0.002511
945,Tin Vodopivec - Peti element | Klub Bazen Kranj,uspešen turneja rocka roll tina vodopivec vrač...,2019-12-07 20:00:00,360,1.001721e+14,5,0.002604
5200,Tin Vodopivec - Peti element,uspešen turneja rocka roll tina vodopivec vrač...,2019-12-27 20:00:00,106,5.347883e+14,5,0.002604
6681,Tin Vodopivec - Peti element,uspešen turneja rocka roll tina vodopivec vrač...,2020-02-08 20:00:00,263,1.550169e+14,5,0.002604
6785,Tin Vodopivec - Peti element,uspešen turneja rocka roll tina vodopivec vrač...,2020-02-01 20:00:00,123,1.116760e+14,5,0.002604
5278,Reporter Milan – Swingerji,predstava srečen konec država štajerski republ...,2019-08-16 20:00:00,377,5.187510e+14,5,0.003504
6509,Nika Gorič - sopran,zaslišati trajati vstopnica društvo arsan jadr...,2019-07-27 19:30:00,128,1.382778e+14,5,0.003671
1350,Pank tura z Esadom Babačićem (kolesarska),tokrat leninov park trg revolucija potekati ko...,2019-08-16 17:00:00,192,1.377444e+15,5,0.004037
3086,"Gregor Strniša, Maruša Kink: ŽABE",premiera predstava ponovitev januar močvirje b...,2019-12-26 18:00:00,343,1.392780e+14,5,0.004982


Cluster 6:


,naziv,opis,datum_od,stevilo,sifra_prizorisca,cluster,center_distance
6679,Zmelkoow v Podzemlju Pece,vabiti tradicionalen koncert podzemlje pec org...,2019-11-30 17:00:00,202,6.728455e+14,6,0.003271
5272,Nočna smuka z apres ski zabavo za dame,nočen smukati smučišče košut jasa uniorček zab...,2020-03-07 17:00:00,179,1.710468e+11,6,0.004658
6081,Spoznavni ŽUR Medicinske fakultete / 10.10. / ...,končen misliti nov študijski leto leto težek p...,2019-10-10 23:00:00,775,2.373029e+14,6,0.004899
7201,Veselica Bašelj - Zvita Feltna,turističen društvo bašelj predstavljati veseli...,2019-08-17 19:00:00,962,2.864456e+15,6,0.006026
5410,Šagra Guoštovca 2019 & Šagra v Maškarah,petek malonogometen turnir otvoritev šagor obl...,2019-08-02 18:00:00,472,1.602198e+15,6,0.006273
1956,SG FEST,esga festi vabiti tradicionalen glasben prired...,2019-09-20 20:00:00,1152,2.561733e+15,6,0.006572
1957,SG FEST,esga festi vabiti tradicionalen glasben prired...,2019-09-20 20:00:00,929,2.561733e+15,6,0.006572
3166,SHOD KAL // 2019,shod kal petek zvit felten zvit felten sobota ...,2019-08-23 21:00:00,1050,2.932229e+15,6,0.006658
5488,JAN Plestenjak,žura zamuditi rezervirati sobota videti vrsta,2019-11-16 20:00:00,1009,2.055271e+15,6,0.007073
4498,Z ansamblom Biseri v veseli december!,leto zavraški gasilec pripravljati pričakati v...,2019-11-23 20:00:00,512,1.106904e+14,6,0.007121
